In [7]:
required_degrees = ["B.Tech"]
candidate_degree = "BDS in Dental"
required_specializations= ["Computer Science", "Electronics", "Software Engineering"]
spec_a=required_specializations

import re

def extract_degree_only(text):
    """
    Extracts the degree part from a string like "BE in Computer Science".
    Returns the degree in uppercase.
    """
    match = re.match(r"([A-Za-z\.]+)", text.strip())
    if match:
        return match.group(1).upper()
    return None



def extract_specialization(text):
    # Normalize spaces
    text = text.strip()

    # Regex pattern to capture text after 'in' or 'of' (e.g., 'BDS in Dental')
    match = re.search(r'\b(?:in|of)\s+([A-Za-z &\-]+)', text, re.IGNORECASE)

    if match:
        specialization = match.group(1).strip()
        return specialization
    return None

degree = extract_degree_only(candidate_degree)
specialization=extract_specialization(candidate_degree)
print("Degree: "+ degree+ "   Specialization: "+specialization)

Degree: BDS   Specialization: Dental


###EDUCATION SCORE CALCULATION

1) SPECIALIZATION SCORE

In [19]:
from sentence_transformers import SentenceTransformer, util

# Load the sentence transformer model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def compare_specializations(specs_a, spec_b):
    # Ensure specs_a is a list
    if isinstance(specs_a, str):
        specs_a = [specs_a]

    # Encode all specializations in specs_a and spec_b
    embeddings_a = model.encode(specs_a, convert_to_tensor=True)
    embedding_b = model.encode(spec_b, convert_to_tensor=True)

    # Compute cosine similarities
    cosine_scores = util.pytorch_cos_sim(embeddings_a, embedding_b).squeeze()

    # Find best score and corresponding specialization
    best_index = torch.argmax(cosine_scores).item()
    best_score = cosine_scores[best_index].item()
    best_spec = specs_a[best_index]

    # Convert to similarity percentage and cap at 100
    similarity_percentage = round(min(best_score * 100 + 20, 100))

    return {
        "best_score": similarity_percentage,
        "matched_specialization": best_spec
    }

# Example usage


result = compare_specializations(spec_a, specialization)
spec_score_=result['best_score']
print(f"Best match for '{specialization}' is '{result['matched_specialization']}' with score: {result['best_score']}%")


Best match for 'Dental' is 'Electronics' with score: 46%


2) DEGREE SCORE CALCULATION

In [15]:
#LIST ALL EQUIVALENT DEGREES FIRST
equivalent_degrees = {
    "B.Tech": ["Bachelor of Technology", "B.E.", "Bachelor of Engineering"],
    "M.Tech": ["Master of Technology", "M.E.", "Master of Engineering"],
    "B.Sc.": ["Bachelor of Science"],
    "M.Sc.": ["Master of Science"],
    "B.Com.": ["Bachelor of Commerce"],
    "M.Com.": ["Master of Commerce"],
    "B.A.": ["Bachelor of Arts"],
    "M.A.": ["Master of Arts"],
    "BBA": ["Bachelor of Business Administration"],
    "MBA": ["Master of Business Administration", "PGDM"],
    "BCA": ["Bachelor of Computer Applications"],
    "MCA": ["Master of Computer Applications"],
    "LL.B.": ["Bachelor of Laws"],
    "LL.M.": ["Master of Laws"],
    "MBBS": ["Bachelor of Medicine, Bachelor of Surgery"],
    "MD": ["Doctor of Medicine"],
    "BDS": ["Bachelor of Dental Surgery"],
    "B.Pharm": ["Bachelor of Pharmacy"],
    "M.Pharm": ["Master of Pharmacy"],
    "B.Arch": ["Bachelor of Architecture"],
    "M.Arch": ["Master of Architecture"],
    "B.Ed.": ["Bachelor of Education"],
    "M.Ed.": ["Master of Education"],
    "Ph.D.": ["Doctor of Philosophy"]
}

from sentence_transformers import SentenceTransformer, util
import torch

# Load the sentence-transformer model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def get_degree_score_with_nlp(required_degrees, candidate_degree, equivalent_degrees, threshold=0.75):
    normalized_required = [deg.strip().lower() for deg in required_degrees]
    candidate_degree_norm = candidate_degree.strip().lower()

    # Handle generic UG requirement
    if any(req in {"ug", "undergraduate", "any degree"} for req in normalized_required):
        return 100

    # Check for exact match or known equivalence
    for req_degree in required_degrees:
        for base, equivalents in equivalent_degrees.items():
            if req_degree == base or req_degree in equivalents:
                if candidate_degree == base or candidate_degree in equivalents:
                    return 100

    # NLP fallback: compute similarity
    best_score = 0
    for req_degree in required_degrees:
        emb1 = model.encode(req_degree, convert_to_tensor=True)
        emb2 = model.encode(candidate_degree, convert_to_tensor=True)
        score = util.pytorch_cos_sim(emb1, emb2).item()
        if score > best_score:
            best_score = score

    # Normalize cosine similarity [0.3, threshold] → [0, 90]
    if best_score < 0.3:
        return 0
    elif best_score > threshold:
        return 90
    return round((best_score - 0.3) / (threshold - 0.3) * 90)



score = get_degree_score_with_nlp(required_degrees, degree , equivalent_degrees)
print("Required Degree: " + str(required_degrees) + " Candidate Degree: " + degree)
print("Degree Score:", score)




Required Degree: ['B.Tech'] Candidate Degree: BDS
Degree Score: 21


In [21]:
education_score=spec_score_+score  #normln fun code as of now
print(education_score)

67


###JOB ROLE SCORE CALCULATION

In [22]:
import re
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# List of generic titles
GENERIC_TITLES = [
    "developer", "engineer", "specialist", "expert", "consultant", "analyst", "associate",
    "technician", "architect", "administrator", "manager", "lead", "director", "coordinator",
    "officer", "supervisor", "operator", "executive", "intern", "assistant",
    "junior", "senior", "principal", "head", "chief", "staff", "entry", "mid",
    "experienced", "trainee",
     "hardware", "systems", "support", "infrastructure", "solutions",
    "operations", "it", "technology", "product", "business", "digital",
    "corporate", "services"
]

def clean_role_title(role: str) -> str:
    """
    Remove generic words from the role title and extra spaces.
    """
    role = role.lower()
    # Remove generic words using regex word boundaries
    pattern = r'\b(' + '|'.join(map(re.escape, GENERIC_TITLES)) + r')\b'
    cleaned = re.sub(pattern, '', role)
    # Remove extra spaces
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    return cleaned

def normalize_score(raw_score, low=0.3, high=0.9):
    if raw_score < low:
        return 0
    elif raw_score > high:
        return 100
    return round((raw_score - low) / (high - low) * 100)

def get_role_similarity_score(role1: str, role2: str) -> int:
    # Clean role titles
    role1_clean = clean_role_title(role1)
    role2_clean = clean_role_title(role2)


    # Add minimal context
    role1_text = f"The job title is {role1_clean}."
    role2_text = f"The job title is {role2_clean}."



    emb1 = model.encode(role1_text, convert_to_tensor=True)
    emb2 = model.encode(role2_text, convert_to_tensor=True)

    cosine_sim = util.pytorch_cos_sim(emb1, emb2).item()

    return normalize_score(cosine_sim)


In [24]:
import pandas as pd
test_cases = [
    ("React Developer", "Node JS Developer"),            # related frontend/backend dev
    ("ML Engineer", "Data Scientist"),                    # closely related AI/data roles
    ("Mechanical Engineer", "Frontend Developer"),        # very different fields
    ("Doctor", "Scrum Master"),                           # unrelated medical vs management
    ("Data Analyst", "Business Intelligence Analyst"),   # very related analysis roles
    ("Project Manager", "Scrum Master"),                  # similar project management roles
    ("Sales Manager", "Marketing Manager"),               # related but different specialties
    ("Graphic Designer", "UX Designer"),                  # related design roles
    ("DevOps Engineer", "Site Reliability Engineer"),    # closely related IT ops roles
    ("Accountant", "Financial Analyst"),                  # finance roles with overlap
    ("Civil Engineer", "Structural Engineer"),            # closely related engineering roles
    ("Content Writer", "Copywriter"),                      # very similar writing roles
    ("HR Manager", "Recruiter"),                           # related HR roles
    ("Teacher", "Professor"),                              # related education roles
    ("Pharmacist", "Chemist"),                             # related science/health roles
    ("Backend Developer", "Database Administrator"),      # related but distinct IT roles
    ("QA Engineer", "Software Tester"),                    # highly similar roles
    ("Customer Support", "Technical Support"),             # overlapping support roles
    ("Chef", "Restaurant Manager"),                        # related but different restaurant jobs
    ("Electrician", "Plumber"),                            # both trades but different skills
    ("Frontend Developer", "React Developer"),                 # high
    ("Backend Developer", "Java Developer"),                   # high
    ("Full Stack Developer", "Node.js Developer"),             # high
    ("DevOps Engineer", "Cloud Engineer"),                     # high
    ("System Administrator", "Network Engineer"),              # moderate
    ("Database Administrator", "Data Engineer"),               # moderate
    ("Data Scientist", "Data Analyst"),                        # high
    ("Machine Learning Engineer", "Deep Learning Engineer"),   # high
    ("Product Manager", "Business Analyst"),                   # moderate
    ("QA Engineer", "Software Tester"),                        # high
    ("Security Engineer", "Penetration Tester"),               # high
    ("Cloud Architect", "AWS Solutions Architect"),            # high
    ("Frontend Developer", "UI Developer"),                    # high
    ("UX Designer", "UI Designer"),                            # high
    ("DevOps Engineer", "Site Reliability Engineer"),          # high
    ("IT Support Specialist", "Help Desk Technician"),         # high
    ("Network Security Engineer", "Cybersecurity Analyst"),    # high
    ("Java Developer", "C++ Developer"),                       # moderate
    ("Python Developer", "Data Engineer"),                     # moderate
    ("Business Intelligence Analyst", "Data Analyst"),         # high
    ("ETL Developer", "Data Engineer"),                        # high
    ("Scrum Master", "Agile Coach"),                           # high
    ("Project Manager", "Delivery Manager"),                   # high
    ("Cloud Engineer", "Azure Engineer"),                      # high
    ("React Developer", "Angular Developer"),                  # moderate
    ("Software Developer", "Software Engineer"),               # high
    ("Database Administrator", "MySQL Developer"),             # high
    ("iOS Developer", "Android Developer"),                    # moderate
    ("Technical Lead", "Engineering Manager"),                 # high
    ("Solutions Architect", "Enterprise Architect")            # moderate

]

for r1, r2 in test_cases:
    score = get_role_similarity_score(r1, r2)
    print(f"Similarity({r1!r}, {r2!r}) = {score}")

results = []
for role1, role2 in test_cases:
    score = get_role_similarity_score(role1, role2)
    results.append({"Role 1": role1, "Role 2": role2, "Similarity Score": score})




Similarity('React Developer', 'Node JS Developer') = 67
Similarity('ML Engineer', 'Data Scientist') = 70
Similarity('Mechanical Engineer', 'Frontend Developer') = 41
Similarity('Doctor', 'Scrum Master') = 15
Similarity('Data Analyst', 'Business Intelligence Analyst') = 49
Similarity('Project Manager', 'Scrum Master') = 38
Similarity('Sales Manager', 'Marketing Manager') = 85
Similarity('Graphic Designer', 'UX Designer') = 92
Similarity('DevOps Engineer', 'Site Reliability Engineer') = 46
Similarity('Accountant', 'Financial Analyst') = 85
Similarity('Civil Engineer', 'Structural Engineer') = 45
Similarity('Content Writer', 'Copywriter') = 86
Similarity('HR Manager', 'Recruiter') = 64
Similarity('Teacher', 'Professor') = 83
Similarity('Pharmacist', 'Chemist') = 69
Similarity('Backend Developer', 'Database Administrator') = 69
Similarity('QA Engineer', 'Software Tester') = 80
Similarity('Customer Support', 'Technical Support') = 57
Similarity('Chef', 'Restaurant Manager') = 80
Similarity(